In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import spacy
from spacy.language import Language
from spacy import displacy
import time
import glob
import re
import math
import statistics
import os
import json
import calendar
import holidays
from pathlib import Path
from datetime import date
from datetime import datetime
import pandas as pd
import numpy as np
import collections
import hashlib
from dateutil.parser import parse
import shutil
import ast
from io import StringIO
import requests

In [2]:
import spacy
from spacy.language import Language
from spacy import displacy
import time

@Language.component("newsent")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        #print(token.text, token.text in ("’s", "'s"))
        if token.text.upper() in (";", "--", "\n\n", "\n", "QUARTERLY", "STORY", "\n\n\n\n", "\n\n\n"):
            #print("Detected:", token.text)
            doc[token.i].is_sent_start = True
    return doc

#spacy.require_gpu()
nlp = spacy.load("../../Summary/NER/RelateEntity/train/model-best-local")
nlp.add_pipe('sentencizer')
nlp.add_pipe('newsent', name="newsent", last=True)

c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\spacy_transformers\pipeline_component.py:406: UserWarning: Automatically converting a transformer component from spacy-transformers v1.0 to v1.1+. If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spacy-transformers version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


<function __main__.set_custom_boundaries(doc)>

In [47]:
def isScanningRqd(pfile):
    with open(pfile, 'r', encoding = "utf-8") as fp:
        for l_no, line in enumerate(fp):
            if "NOPAD***" in line:
                return False
    return True

In [3]:
def getSentences(inputfile, nlp, text=None):
    if(not text):
        with open(inputfile, 'r', encoding="utf-8") as f:
            text = f.read()

    doc = nlp(text)
    sentences = [str(sent).strip() for sent in doc.sents]

    #print(len(sentences))
    return(sentences)

In [4]:
rplStr = ["PG*** ", "ED*** ", "SCHQ*** ", "SCBQ*** ", "SCBF*** ", "SCHF*** ", "SCG*** ", "GF*** ", "GQ*** ", "SC*** ", "NOPAD*** "]

def preProcessSent(line):
    for s in rplStr:
        line = line.replace(s, "")
    if("TBLST***" in line or "TBLET***" in line or "CS***" in line or line == "\n" or line == "\n\n" or "https://finance.yahoo.com" in line):
        return(line, False)
    line = line.replace("\n", "")
    if(line == None or line == ""):
        return(line, False)
    return(line, True)

In [5]:
import hashlib
def getHash(sent):
    hash_object = hashlib.sha1(sent.upper().encode("UTF-8"))
    hex_dig = hash_object.hexdigest()
    return(hex_dig)

In [43]:
def getRelationFromER(sym):
    entityFile = "../../Summary/entities/"+sym+"-ENTITIES.json"
    print(entityFile)
    #search = "Q\d+-[0-9][0-9][0-9][0-9]|ALL-[0-9][0-9][0-9][0-9]"
    search = "Q\d+-[0-9][0-9][0-9][0-9]"
    print(search)
    cnt = 0
    sentER = dict()
    with open(entityFile, encoding="utf-8") as f:
        entity = json.load(f)
    if(entity):
        for key in entity[sym].keys():
            if(re.search(search, key)):
                if("ALL" not in key):
                    cnt = cnt + 1
                #if(cnt == 1):
                #    continue
                if(cnt > 5):
                    break
                for ent in entity[sym][key]:
                    if("PREV" not in ent and type(entity[sym][key][ent]) == dict and "RELATION" in entity[sym][key][ent] and len(entity[sym][key][ent]["RELATION"]) > 0):
                        for r in entity[sym][key][ent]["RELATION"]:
                            newent = ent.replace("-GUIDEFULL","")
                            newent = newent.replace("-GUIDE","")
                            newent = newent.replace("-FULLYEAR","")
                            newent = re.sub("-"+search, "", newent)
                            sent = r.split(":::")[2]
                            sent = sent.strip()
                            hexDig = getHash(sent)
                            relation = "METRIC:"+newent+","
                            if hexDig not in sentER:
                                sentER[hexDig] = dict()
                            else:
                                if("RELATION" in sentER[hexDig] and relation in sentER[hexDig]["RELATION"][0]):
                                    print("SAME RELATION " + relation + " ALREADY PRESENT FOR " + sent)
                                    continue
                            if("TEXT-METRICTYPE" in entity[sym][key][ent] and entity[sym][key][ent]["TEXT-METRICTYPE"] == "OPMETRIC"):
                                relation = relation + "METRICTYPE:OPMETRIC,"
                            else:
                                relation = relation + "METRICTYPE:REGULAR,"
                            if("RTEXT-MONEY" in entity[sym][key][ent] and "RTEXT-RANGEM1" in entity[sym][key][ent]):
                                relation = relation + "MONEY:"+entity[sym][key][ent]["RTEXT-MONEY"]+",RANGEM1:"+entity[sym][key][ent]["RTEXT-RANGEM1"]+",RELATION:KVRANGE"
                            elif("RTEXT-CD" in entity[sym][key][ent] and "RTEXT-RANGEM1" in entity[sym][key][ent]):
                                relation = relation + "CD:"+entity[sym][key][ent]["RTEXT-CD"]+",RANGEM1:"+entity[sym][key][ent]["RTEXT-RANGEM1"]+",RELATION:KVRANGE"
                            elif("RTEXT-PCT" in entity[sym][key][ent] and "RTEXT-RANGEM1" in entity[sym][key][ent]):
                                relation = relation + "PCT:"+entity[sym][key][ent]["RTEXT-PCT"]+",RANGEM1:"+entity[sym][key][ent]["RTEXT-RANGEM1"]+",RELATION:KVRANGE"
                            elif("RTEXT-MONEY" in entity[sym][key][ent]):
                                relation = relation + "MONEY:"+entity[sym][key][ent]["RTEXT-MONEY"]+",RELATION:KV"
                            elif("RTEXT-CD" in entity[sym][key][ent]):
                                relation = relation + "CD:"+entity[sym][key][ent]["RTEXT-CD"]+",RELATION:KV"
                            elif("RTEXT-PCT" in entity[sym][key][ent]):
                                relation = relation + "PCT:"+entity[sym][key][ent]["RTEXT-PCT"]+",RELATION:KV"
                            #print(key, sent+"\t"+relation, hexDig)
                            sentER[hexDig]["ORIGSENT"] = sent
                            sentER[hexDig]["KEY"] = key
                            #sentER[hexDig]["PROPERTY"] = "REGULAR"
                            if("RELATION" not in sentER[hexDig]):
                                sentER[hexDig]["RELATION"] = list()
                            sentER[hexDig]["RELATION"].append(relation)
    #print(sentER)
    return(sentER)

In [7]:
def getFileSortTimestamp(csym, src="REFINED"):
    if(src == "POSTREFINED"):
        path = "../../Summary/PostRefined"
    else:
        path = "../../Summary/Refined"
    files = glob.glob(path+"/"+csym+"/*")   
    files.sort(reverse=True, key=os.path.getmtime)
    return(files)

In [8]:
def getOrgData(org):
    orgDataPath = "../../Summary/orgData/"+org+".txt"
    file = Path(orgDataPath)
    if file.is_file():
        #print(True)
        with open(orgDataPath) as f:
            data = json.load(f)
        #print(data)
        return data
    return None

In [9]:
def getOrgAttr(orgData, attr):
    if not orgData:
        return None
    asplit = attr.split("|")
    parent = asplit[0]
    if "SOURCE" in orgData[parent]:
        src = orgData[parent]["SOURCE"]
        if src == "YH" or (parent == "ORGPROFILE" and src == "AD"):
            p = orgData
            for i in range(0, len(asplit)):
                if asplit[i] not in p:
                    return None
                p = p[asplit[i]]
            #print(p)
            return(p)
    return None

In [10]:
def getQtrs(org, edate, fynd):
    #print(edate)
    dateObj = datetime.strptime(edate, '%Y-%m-%d')
    #print(dateObj.month)
    emonth = dateObj.month
    eyr = dateObj.year
    k = 1
    qend = fynd
    qstart = 0
    year = eyr
    #print(year)
    qtrs = dict()
    while(k!=5):
        #print(k)
        qstart = (qend + 1)%12
        qend = (qend + 3)%12
        if qstart == 0:
            qstart = 12
        if qend == 0:
            qend = 12
        if((emonth >= qstart and emonth <= qend) or (qstart > qend and (emonth >= qstart or emonth <= qend))):
            #print("CQTR:", "Q"+str(k))
            qtrs["CQTR"] = "Q"+str(k)
            qtrs["GQTR"] = "Q"+str(k) # Guidance qtr is same as current qtr
            qtrs["CYR"] = str(year)
            qtrs["GYR"] = str(year) # Guidance year same as current year
            eqtr = (k-1)%4
            if(eqtr == 0):
                eqtr = 4
            nqtr = (k+1)%4
            if(nqtr == 0):
                nqtr = 4
            pqtr = (k-2)%4
            if(pqtr == 0):
                pqtr = 4
            qtrs["EQTR"] = "Q"+str(eqtr)
            qtrs["PQTR"] = "Q"+str(pqtr)
            #print("EQTR:", "Q"+str(eqtr))
            #print("PQTR:", "Q"+str(pqtr))
            #print("NQTR:", "Q"+str(nqtr))
        #print("Q"+str(k), "START", qstart)
        #print("Q"+str(k), "END", qend)
        qtr = "Q"+str(k)
        if (qtr not in qtrs):
            qtrs[qtr] = dict()
        qtrs[qtr]["START"] = qstart
        qtrs[qtr]["SM"] = calendar.month_abbr[qstart].upper()
        qtrs[qtr]["END"] = qend
        qtrs[qtr]["EM"] = calendar.month_abbr[qend].upper()
        #print("Q"+str(k), "YEAR", year)
        k = k + 1
    
    #print(eyr)   
    if("EQTR" in qtrs and "CQTR" in qtrs and "PQTR" in qtrs):
        eqtrEnd = qtrs[qtrs["EQTR"]]["END"]
        cqtrEnd = qtrs[qtrs["CQTR"]]["END"]
        pqtrEnd = qtrs[qtrs["PQTR"]]["END"]
        #print(eqtrEnd, cqtrEnd, pqtrEnd)
        qtrs["EYR"] = str(eyr)
        qtrs["PYR"] = str(eyr)
        #print(eqtrEnd, cqtrEnd, qtrs[qtrs["CQTR"]]["START"], qtrs[qtrs["CQTR"]]["END"])
        if(eqtrEnd > cqtrEnd):
            if(qtrs[qtrs["CQTR"]]["START"] > qtrs[qtrs["CQTR"]]["END"] and emonth > fynd):
                gyr = year + 1
                qtrs["GYR"] = str(gyr)
                qtrs["EYR"] = str(gyr - 1)
            else:
                qtrs["EYR"] = str(eyr - 1)
                qtrs["PYR"] = str(eyr - 1)
        if(pqtrEnd > eqtrEnd):
            qtrs["PYR"] = str(int(qtrs["EYR"]) - 1)
    if(emonth <= fynd):
        qtrs["CFYR"] = str(year)
    else:
        qtrs["CFYR"] = str(year + 1)
    qtrs["FYR"] = qtrs["CFYR"]
    if(qtrs[qtrs["EQTR"]]["END"] <= fynd and qtrs["EQTR"] > qtrs["CQTR"]):
        qtrs["FYR"] = str(int(qtrs["FYR"]) - 1)
    qtrs["EFYR"] = qtrs["FYR"]
    qtrs["PFYR"] = qtrs["FYR"]
    if(qtrs["PQTR"] > qtrs["EQTR"]):
        qtrs["PFYR"] = str(int(qtrs["FYR"]) - 1)
    qtrs["GFYR"] = qtrs["CFYR"]
    #qtrs["EYR"] = qtrs["FYR"]
    #qtrs["CYR"] = qtrs["CFYR"]
    #qtrs["PYR"] = qtrs["FYR"]
    qtrs["CYR"] = qtrs["GYR"]
    #print(qtrs)
    return (qtrs)

In [37]:
trainDir = "../../Summary/DATA/ENTITYRELATION/Train"
devDir = "../../Summary/DATA/ENTITYRELATION/Dev"

def scanER(csym, estr, sentER, infile, outfile=None):
    infilePath = Path(infile)
    if not infilePath:
        print("Input File {} does not exist".format(infile))
        return
    basefile = os.path.basename(infile)
    filename = os.path.splitext(basefile)[0]
    
    outfilePath = None
    of = None
    if outfile:
        outfilePath = Path(outfile)
        if outfilePath.is_file():
            print("Output File {} already exists".format(outfile))
            return
        
        of = open(outfilePath, "w", encoding = "utf-8")
        of.write("filename\tSentence1\tSentence2\n")
        
    with open(infilePath, "r", encoding = "utf-8") as f:
        line = f.readline()
        #print(line)
        while line:
            #print(line)
            if("ED***" in line):
                break
            elif("CS***" in line or "TBLST***" in line or "TBLET***" in line):
                #print(line+"\t"+"RELATION:None")
                line = f.readline()
                continue
            #elif("PG***" in line or "NOPAD***" in line or ("***" not in line)):
            else:
                sentences = getSentences(None, nlp, line)
                for l in sentences:
                    #print(l)
                    pline, isProcess = preProcessSent(l)
                    if not isProcess:
                        continue
                    text1 = list()
                    text1.append(l)
                    nerl = list()
                    for doc in nlp.pipe(text1, disable=["tagger"]):
                        nerl = ([(ent.text, ent.label_) for ent in doc.ents])
                        #print("NER", nerl)
                    hexDig = getHash(pline)
                    if hexDig in sentER and "KEY" in sentER[hexDig] and sentER[hexDig]["KEY"] == estr:
                        print(l+"\t"+"NER:"+str(nerl)+"###RELATIONS:"+str(sentER[hexDig]["RELATION"]))
                        of.write(filename+"\t"+l+"\t"+"NER:"+str(nerl)+"###RELATIONS:"+str(sentER[hexDig]["RELATION"])+"\n\n\n")
                    else:
                        print(l+"\t"+"NER:"+str(nerl)+"###RELATIONS:"+str(list()))
                        of.write(filename+"\t"+l+"\t"+"NER:"+str(nerl)+"###RELATIONS:"+str(list())+"\n\n\n")
            line = f.readline()
    if(of):
        of.close()

In [48]:
def createTrainingData(csym):
    files = getFileSortTimestamp(csym)
    print(files)
    pfiles = getFileSortTimestamp(csym, "POSTREFINED")
    print(pfiles)

    sentER = getRelationFromER(csym)

    orgData = getOrgData(csym)
    if orgData:
        val = getOrgAttr(orgData, "ORGPROFILE|FiscalYearEnd")

    cnt = 0
    cdate = "2023-06-30"
    for file in files:
        if not isScanningRqd(file):
            continue
        cnt = cnt + 1
        if(cnt > 5):
            break
        basefile = os.path.basename(file)
        filename = os.path.splitext(basefile)[0]
        outfile = filename+"_ER.tsv"
        outfile = trainDir + "/" + outfile
        #print(filename)
        edate = filename.split("_")[1]
        rtype = filename.split("_")[0]
        estr = None
        if orgData:
            if(val):
                dobj = datetime.strptime(val, "%B")
                fynd = (dobj.month)
                qtrs = getQtrs(csym, edate, fynd)
                eqtr = qtrs["EQTR"]
                efyr = qtrs["EFYR"]
                estr = eqtr+"-"+efyr
        if(estr):
            nedate = time.strptime(edate, "%Y-%m-%d")
            ncdate = time.strptime(cdate, "%Y-%m-%d")
            if(nedate > ncdate):
                nfile = "../../Summary/PostRefined/"+csym+"/"+basefile
                print(edate, rtype, estr, nfile, outfile)
            else:
                nfile = file
                print(edate, rtype, estr, nfile, outfile)
            scanER(csym, estr, sentER, nfile, outfile)

In [50]:
csym = "APPN"
createTrainingData(csym)

['../../Summary/Refined/APPN\\APPN_2023-05-09_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2023-08-03_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2023-02-16_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2022-11-03_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2022-08-04_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2022-05-06_EP_YH.txt']
['../../Summary/PostRefined/APPN\\APPN_2023-08-03_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2023-05-09_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2023-02-16_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2022-11-03_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2022-08-04_EP_YH.txt']
../../Summary/entities/APPN-ENTITIES.json
Q\d+-[0-9][0-9][0-9][0-9]
2023-05-09 APPN Q1-2023 ../../Summary/Refined/APPN\APPN_2023-05-09_EP_YH.txt ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-05-09_EP_YH_ER.tsv
GAAP Net Loss Per Share is $(0.51) in first quarter 2023 @@@	NER:[('GAAP Net Loss Per Share', 'METRIC'), ('$(0.51)', 'MONEY'), ('first qu

PG*** Balance sheet and cash flows: As of March 31, 2023, Appian had total cash, cash equivalents, and investments of $254.5 million.	NER:[('March 31, 2023', 'DATE'), ('Appian', 'ORG'), ('cash', 'METRIC'), ('cash equivalents', 'METRIC'), ('$254.5 million', 'MONEY')]###RELATIONS:['METRIC:CASH AND EQUIVALENTS,METRICTYPE:REGULAR,MONEY:$254.5MN,RELATION:KV']
Net cash used by operating activities was $(25.3) million for the three months ended March 31, 2023 compared to $(20.6) million of net cash used by operating activities for the same period in 2022.	NER:[('cash used by operating activities', 'METRIC'), ('$(25.3) million', 'MONEY'), ('March 31, 2023', 'DATE'), ('$(20.6) million', 'MONEY'), ('cash used by operating activities', 'METRIC'), ('2022', 'YEAR')]###RELATIONS:['METRIC:OPERATING CASH FLOW,METRICTYPE:REGULAR,MONEY:-$25.3MN,RELATION:KV']
PG***	NER:[]###RELATIONS:[]
Story continues.	NER:[]###RELATIONS:[]
PG*** A reconciliation of GAAP to non-GAAP financial measures has been provided 

Cloud subscription revenue retention rate is 115% as of June 30, 2023 .	NER:[('Cloud', 'MISC'), ('subscription revenue', 'METRIC'), ('retention rate', 'METRIC'), ('115%', 'PCT'), ('June 30, 2023', 'DATE')]###RELATIONS:['METRIC:NET DOLLAR EXPANSION,METRICTYPE:OPMETRIC,PCT:115.0%,RELATION:KV']
PG*** GAAP operating loss is $(40.7) million.	NER:[('GAAP', 'METRIC'), ('operating loss', 'METRIC'), ('$(40.7) million', 'MONEY')]###RELATIONS:['METRIC:GAAP OPERATING INCOME,METRICTYPE:REGULAR,MONEY:-$40.7MN,RELATION:KV']
Non-GAAP operating loss is $(27.1) million .	NER:[('Non-GAAP', 'METRIC'), ('operating loss', 'METRIC'), ('$(27.1) million', 'MONEY')]###RELATIONS:['METRIC:NON-GAAP OPERATING INCOME,METRICTYPE:REGULAR,MONEY:-$27.1MN,RELATION:KV']
PG*** GAAP net loss is $(42.4) million.	NER:[('GAAP', 'METRIC'), ('net loss', 'METRIC'), ('$(42.4) million', 'MONEY')]###RELATIONS:['METRIC:GAAP NET INCOME,METRICTYPE:REGULAR,MONEY:-$42.4MN,RELATION:KV']
GAAP net loss per share is $(.58) for the second qua

PG***	NER:[]###RELATIONS:[]
PG*** Appian Corporation.	NER:[('Appian', 'ORG')]###RELATIONS:[]
PG*** Fourth quarter cloud subscription revenue increased 29% year-over-year to $65.8 million.	NER:[('Fourth quarter', 'CALENDAR'), ('cloud', 'MISC'), ('subscription revenue', 'METRIC'), ('increased', 'CHG'), ('29%', 'PCT'), ('year-over-year', 'CALENDAR'), ('$65.8 million', 'MONEY')]###RELATIONS:[]
Full year cloud subscription revenue increased 32% year-over year to $236.9 million.	NER:[('Full year', 'CALENDAR'), ('cloud', 'MISC'), ('subscription revenue', 'METRIC'), ('increased', 'CHG'), ('32%', 'PCT'), ('year-over year', 'CALENDAR'), ('$236.9 million', 'MONEY')]###RELATIONS:['METRIC:SUBSCRIPTION REVENUE-cloud,METRICTYPE:REGULAR,MONEY:$236.9MN,RELATION:KV']
PG*** MCLEAN, Va., Feb. 16, 2023 (GLOBE NEWSWIRE)	NER:[('MCLEAN', 'LOC'), ('Feb. 16, 2023', 'DATE')]###RELATIONS:[]
-- Appian (Nasdaq: APPN) today announced financial results for the fourth quarter and full year ended December 31, 2022.	NER

PG*** Operating loss and non-GAAP operating loss: GAAP operating loss was $(145.0) million for the full year 2022, compared to $(83.9) million for the full year 2021.	NER:[('Operating loss', 'METRIC'), ('non-GAAP', 'METRIC'), ('operating loss', 'METRIC'), ('GAAP', 'METRIC'), ('operating loss', 'METRIC'), ('$(145.0) million', 'MONEY'), ('full year', 'CALENDAR'), ('2022', 'YEAR'), ('$(83.9) million', 'MONEY'), ('full year', 'CALENDAR'), ('2021', 'YEAR')]###RELATIONS:['METRIC:GAAP OPERATING INCOME,METRICTYPE:REGULAR,MONEY:-$145.0MN,RELATION:KV']
Non-GAAP operating loss was $(83.3) million for the full year 2022, compared to $(43.7) million for the full year 2021.	NER:[('Non-GAAP', 'METRIC'), ('operating loss', 'METRIC'), ('$(83.3) million', 'MONEY'), ('full year', 'CALENDAR'), ('2022', 'YEAR'), ('$(43.7) million', 'MONEY'), ('full year', 'CALENDAR'), ('2021', 'YEAR')]###RELATIONS:['METRIC:NON-GAAP OPERATING INCOME,METRICTYPE:REGULAR,MONEY:-$83.3MN,RELATION:KV']
PG*** Net loss and non-GAAP

PG*** Cloud subscription revenue is expected to be between $294.0 million and $296.0 million, representing year-over-year growth of 24% to 25%.	NER:[('Cloud', 'MISC'), ('subscription revenue', 'METRIC'), ('$294.0 million', 'MONEY'), ('$296.0 million', 'MONEY'), ('year-over-year', 'CALENDAR'), ('growth', 'CHG'), ('24%', 'PCT'), ('25%', 'PCT')]###RELATIONS:['METRIC:SUBSCRIPTION REVENUE-Cloud,METRICTYPE:REGULAR,MONEY:$294.0MN,RANGEM1:$296.0MN,RELATION:KVRANGE']
PG*** Total revenue is expected to be between $530.0 million and $535.0 million, representing a year-over-year increase of 13% to 14%.	NER:[('revenue', 'METRIC'), ('$530.0 million', 'MONEY'), ('$535.0 million', 'MONEY'), ('year-over-year', 'CALENDAR'), ('increase', 'CHG'), ('13%', 'PCT'), ('14%', 'PCT')]###RELATIONS:['METRIC:REVENUE,METRICTYPE:REGULAR,MONEY:$530.0MN,RANGEM1:$535.0MN,RELATION:KVRANGE']
PG*** Adjusted EBITDA loss is expected to be between $(75.0) million and $(70.0) million.	NER:[('Adjusted EBITDA loss', 'METRIC'), (

PG*** Adjusted EBITDA: Adjusted EBITDA loss was $(22.9) million, compared to adjusted EBITDA loss of $(12.0) million for the third quarter of 2021.	NER:[('Adjusted EBITDA', 'METRIC'), ('Adjusted EBITDA loss', 'METRIC'), ('$(22.9) million', 'MONEY'), ('adjusted EBITDA loss', 'METRIC'), ('$(12.0) million', 'MONEY'), ('third quarter', 'CALENDAR'), ('2021', 'YEAR')]###RELATIONS:['METRIC:ADJUSTED EBITDA,METRICTYPE:REGULAR,MONEY:-$22.9MN,RELATION:KV']
PG*** Balance sheet and cash flows: As of September 30, 2022, Appian had total unencumbered cash and investments of $92.7 million.	NER:[('September 30, 2022', 'DATE'), ('Appian', 'ORG'), ('cash', 'METRIC'), ('$92.7 million', 'MONEY')]###RELATIONS:['METRIC:CASH AND EQUIVALENTS,METRICTYPE:REGULAR,MONEY:$92.7MN,RELATION:KV']
Net cash used in operating activities was $(43.7) million for the three months ended September 30, 2022 compared to $(25.1) million of net cash used in operating activities for the same period in 2021.	NER:[('cash used in oper

SCHQ*** Second Quarter 2022 Financial Highlights:.	NER:[('SCHQ', 'ROLE'), ('Second Quarter', 'CALENDAR'), ('2022', 'YEAR')]###RELATIONS:[]
PG*** Revenue: Cloud subscription revenue was $57.1 million for the second quarter of 2022, up 34% compared to the second quarter of 2021.	NER:[('Cloud', 'MISC'), ('subscription revenue', 'METRIC'), ('$57.1 million', 'MONEY'), ('second quarter', 'CALENDAR'), ('2022', 'YEAR'), ('up', 'CHG'), ('34%', 'PCT'), ('second quarter', 'CALENDAR'), ('2021', 'YEAR')]###RELATIONS:['METRIC:SUBSCRIPTION REVENUE-Cloud,METRICTYPE:OPMETRIC,MONEY:$57.1MN,RELATION:KV']
Total subscriptions revenue increased 35% year-over-year to $76.7 million.	NER:[('subscriptions revenue', 'METRIC'), ('increased', 'CHG'), ('35%', 'PCT'), ('year-over-year', 'CALENDAR'), ('$76.7 million', 'MONEY')]###RELATIONS:['METRIC:SUBSCRIPTIONS REVENUE,METRICTYPE:OPMETRIC,MONEY:$76.7MN,RELATION:KV']
Professional services revenue was $33.4 million, an increase of 28% compared to the second quarter of

SCG*** Financial Outlook:.	NER:[('Outlook', 'FC')]###RELATIONS:[]
PG*** As of August 4, 2022, guidance for 2022 is as follows:.	NER:[('August 4, 2022', 'DATE'), ('guidance', 'FC'), ('2022', 'YEAR')]###RELATIONS:[]
GQ*** Third Quarter 2022 Guidance:.	NER:[('GQ', 'METRIC'), ('Third Quarter', 'CALENDAR'), ('2022', 'YEAR'), ('Guidance', 'FC')]###RELATIONS:[]
PG*** Cloud subscription revenue is expected to be between $60.8 million and $61.3 million, representing year-over-year growth of 30% to 31%.	NER:[('Cloud', 'MISC'), ('subscription revenue', 'METRIC'), ('$60.8 million', 'MONEY'), ('$61.3 million', 'MONEY'), ('year-over-year', 'CALENDAR'), ('growth', 'CHG'), ('30%', 'PCT'), ('31%', 'PCT')]###RELATIONS:['METRIC:SUBSCRIPTION REVENUE-Cloud,METRICTYPE:REGULAR,MONEY:$60.8MN,RANGEM1:$61.3MN,RELATION:KVRANGE']
PG*** Total revenue is expected to be between $115.0 million and $117.0 million, representing a year-over-year increase of 24% to 27%.	NER:[('revenue', 'METRIC'), ('$115.0 million', 'MON

In [41]:
entPath = "../../Summary/entities/"
files = glob.glob(entPath+"/*-ENTITIES.json")
if(len(files) > 0):
    for file in (files):
        basefile = os.path.basename(file)
        filename = os.path.splitext(basefile)[0]
        csym = filename.split("-")[0]
        #print(csym)
        createTrainingData(csym)

['../../Summary/Refined/APPN\\APPN_2023-05-09_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2023-08-03_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2023-02-16_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2022-11-03_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2022-08-04_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2022-05-06_EP_YH.txt']
['../../Summary/PostRefined/APPN\\APPN_2023-08-03_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2023-05-09_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2023-02-16_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2022-11-03_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2022-08-04_EP_YH.txt']
../../Summary/entities/APPN-ENTITIES.json
Q\d+-[0-9][0-9][0-9][0-9]
2023-05-09 APPN Q1-2023 ../../Summary/Refined/APPN\APPN_2023-05-09_EP_YH.txt ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-05-09_EP_YH_ER.tsv
Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-05-09_EP_YH_ER.tsv already exists
2023-08-03 APPN Q2-2023 ../../Summary/Pos

PG*** Net loss was $31.1 million, or ($0.29) per share, basic and diluted, compared to net loss of $86.7 million, or ($0.84) per share, basic and diluted, in the third quarter of fiscal 2022.	NER:[('Net loss', 'METRIC'), ('$31.1 million', 'MONEY'), ('($0.29)', 'MONEY'), ('per share', 'METRIC'), ('net loss', 'METRIC'), ('$86.7 million', 'MONEY'), ('($0.84)', 'MONEY'), ('per share', 'METRIC'), ('third quarter', 'CALENDAR'), ('2022', 'YEAR')]###RELATIONS:['METRIC:GAAP-EPS,METRICTYPE:REGULAR,MONEY:-$0.29,RELATION:KV', 'METRIC:GAAP NET INCOME,METRICTYPE:REGULAR,MONEY:-$31.1MN,RELATION:KV']
Non-GAAP net income was $58.7 million, or $0.50 per diluted share, compared to non-GAAP net loss of 8.7 million, or ($0.08) per share, basic and diluted, in the third quarter of fiscal 2022.	NER:[('Non-GAAP', 'METRIC'), ('net income', 'METRIC'), ('$58.7 million', 'MONEY'), ('$0.50', 'MONEY'), ('per diluted share', 'METRIC'), ('non-GAAP', 'METRIC'), ('net loss', 'METRIC'), ('8.7 million', 'CD'), ('($0.08)'

GAAP Gross Margin is 81.74% in second quarter 2023 @@@	NER:[('GAAP', 'METRIC'), ('Gross Margin', 'METRIC'), ('81.74%', 'PCT'), ('second quarter', 'CALENDAR')]###RELATIONS:[]
GAAP Free Cash Flow is $53.4MN second quarter 2023 @@@	NER:[('GAAP', 'METRIC'), ('Free Cash Flow', 'METRIC'), ('$53.4MN', 'MONEY'), ('second quarter', 'CALENDAR')]###RELATIONS:[]
GAAP Free Cash Flow Margin is 20.56% in second quarter 2023 @@@	NER:[('GAAP', 'METRIC'), ('Free Cash Flow', 'METRIC'), ('Margin', 'METRIC'), ('20.56%', 'PCT'), ('second quarter', 'CALENDAR')]###RELATIONS:[]
PG*** Q2 Core Revenue Increased 49% Year-Over-Year.	NER:[('Q2', 'CALENDAR'), ('Core Revenue', 'METRIC'), ('Increased', 'CHG'), ('49%', 'PCT'), ('Year-Over-Year', 'CALENDAR')]###RELATIONS:[]
PG*** Q2 Total Revenue Increased 66% Year-Over-Year.	NER:[('Q2', 'CALENDAR'), ('Revenue', 'METRIC'), ('Increased', 'CHG'), ('66%', 'PCT'), ('Year-Over-Year', 'CALENDAR')]###RELATIONS:[]
PG*** SAN JOSE, Calif., February 02, 2023--(BUSINESS WIRE)--BILL

This includes 11.0 million transactions on our BILL standalone platform, representing an increase of 12% year-over-year, and 9.4 million Divvy card transactions, an increase of 77% year-over-year.	NER:[('11.0 million', 'CD'), ('transactions', 'METRIC'), ('BILL standalone platform', 'MISC'), ('increase', 'CHG'), ('12%', 'PCT'), ('year-over-year', 'CALENDAR'), ('9.4 million', 'CD'), ('Divvy card transactions', 'MISC'), ('increase', 'CHG'), ('77%', 'PCT'), ('year-over-year', 'CALENDAR')]###RELATIONS:[]
PG*** Announced that its Board of Directors approved a share repurchase program with authorization to purchase up to $300 million of BILL’s common stock.	NER:[('Announced', 'CHG'), ('Board of Directors', 'ROLE'), ('$300 million', 'MONEY'), ('BILL’s', 'ORG'), ('stock', 'METRIC')]###RELATIONS:[]
SCG*** Financial Outlook.	NER:[('Outlook', 'FC')]###RELATIONS:[]
PG*** We are providing the following guidance for the fiscal third quarter ending March 31, 2023 and the full fiscal year ending June 3

PG*** Loss from operations was $87.7 million, compared to a loss from operations of $74.2 million in the first quarter of fiscal 2022.	NER:[('Loss from operations', 'METRIC'), ('$87.7 million', 'MONEY'), ('loss from operations', 'METRIC'), ('$74.2 million', 'MONEY'), ('first quarter', 'CALENDAR'), ('2022', 'YEAR')]###RELATIONS:[]
Non-GAAP income from operations was $9.1 million, compared to a non-GAAP loss from operations of $9.2 million in the first quarter of fiscal 2022.	NER:[('Non-GAAP', 'METRIC'), ('income from operations', 'METRIC'), ('$9.1 million', 'MONEY'), ('non-GAAP', 'METRIC'), ('loss from operations', 'METRIC'), ('$9.2 million', 'MONEY'), ('first quarter', 'CALENDAR'), ('2022', 'YEAR')]###RELATIONS:[]
PG*** Net loss was $81.6 million, or ($0.78) per share, basic and diluted, compared to net loss of $74.3 million, or ($0.78) per share, basic and diluted, in the first quarter of fiscal 2022.	NER:[('Net loss', 'METRIC'), ('$81.6 million', 'MONEY'), ('($0.78)', 'MONEY'), ('per

...

PG*** Guidance for non-GAAP income from operations excludes stock-based compensation expense and related employer payroll taxes, amortization expense of acquired intangible assets, amortization of debt discount and issuance costs.	NER:[]###RELATIONS:[]
As a result of the adoption of ASU 2020-06 on August 1, 2022, guidance for non-GAAP net income per share uses the if-converted method to calculate the potentially diluted shares related to the convertible senior notes.	NER:[('August 1, 2022', 'DATE'), ('guidance', 'FC'), ('non-GAAP', 'METRIC'), ('net income per share', 'METRIC')]###RELATIONS:[]
Accordingly, we are required to add back the non-GAAP interest expense to our non-GAAP net income and include approximately 7.63 million shares related to our senior convertible notes.	NER:[('non-GAAP', 'METRIC'), ('net income', 'METRIC'), ('7.63 million', 'CD'), ('shares', 'METRIC')]###RELATIONS:[]
Additionally, we include the anti-dilutive impact of the capped call transactions entered into in c

PG*** Deferred revenue: $1,021.1 million as of July 31, 2022, an increase of 62% year-over-year.	NER:[('Deferred revenue', 'METRIC'), ('$1,021.1 million', 'MONEY'), ('July 31, 2022', 'DATE'), ('increase', 'CHG'), ('62%', 'PCT'), ('year-over-year', 'CALENDAR')]###RELATIONS:[]
PG*** Cash, cash equivalents and short-term investments: $1,731.3 million as of July 31, 2022, an increase of $228.8 million from July 31, 2021.	NER:[('Cash', 'METRIC'), ('cash equivalents', 'METRIC'), ('$1,731.3 million', 'MONEY'), ('July 31, 2022', 'DATE'), ('increase', 'CHG'), ('$228.8 million', 'MONEY'), ('July 31, 2021', 'DATE')]###RELATIONS:['METRIC:CASH AND EQUIVALENTS,METRICTYPE:REGULAR,MONEY:$1731.3MN,RELATION:KV']
SCHF*** Full Year Fiscal 2022 Financial Highlights.	NER:[('SCHF', 'METRIC'), ('Full Year', 'CALENDAR'), ('2022', 'YEAR')]###RELATIONS:[]
PG*** Revenue $1,090.9 million, an increase of 62% year-over-year.	NER:[('Revenue', 'METRIC'), ('$1,090.9 million', 'MONEY'), ('increase', 'CHG'), ('62%', 'PCT

As a result of the adoption of the ASU 2020-06, we are required to use the “if-converted” method to calculate the non-GAAP net income per diluted share related to our senior convertible notes, which assumes conversion at the beginning of the reporting period, with settlement entirely in shares of common stock, unless the results would be anti-dilutive.	NER:[]###RELATIONS:[]
Accordingly, to account for our potentially diluted shares related to our senior convertible notes, we are required to add back the non-GAAP interest expense to our non-GAAP net income and include approximately 7.63 million shares related to our senior convertible notes beginning in the first quarter of fiscal 2023.	NER:[]###RELATIONS:[]
SCG*** Financial Outlook.	NER:[('Outlook', 'FC')]###RELATIONS:[]
GQ*** For the first quarter of fiscal 2023, we expect:.	NER:[('first quarter', 'CALENDAR'), ('2023', 'YEAR'), ('expect', 'FC')]###RELATIONS:[]
PG*** Total revenue of $339 million to $341 million.	NER:[('revenue', 'METR

NOPAD*** SAN JOSE, Calif., May 26, 2022 (GLOBE NEWSWIRE)	NER:[('SAN JOSE', 'LOC'), ('Calif.', 'LOC'), ('May 26, 2022', 'DATE')]###RELATIONS:[]
-- Zscaler, Inc. (Nasdaq: ZS), the leader in cloud security, today announced financial results for its third quarter of fiscal year 2022, ended April 30, 2022.	NER:[('Zscaler', 'ORG'), ('Nasdaq', 'ORG'), ('ZS', 'SYM'), ('leader', 'METRIC'), ('cloud security', 'MISC'), ('announced', 'CHG'), ('third quarter', 'CALENDAR'), ('2022', 'YEAR'), ('April 30, 2022', 'DATE')]###RELATIONS:[]
PG***	NER:[]###RELATIONS:[]
NOPAD*** "Enterprises continue to consolidate point products in favor of our integrated Zero Trust security platform, resulting in larger, multi-year commitments to Zscaler.	NER:[('integrated', 'CHG'), ('Zero Trust security', 'MISC'), ('Zscaler', 'ORG')]###RELATIONS:[]
We delivered 63% revenue growth and 54% billings growth, while also generating a strong free cash flow margin of 15% for the third quarter.	NER:[('63%', 'PCT'), ('revenue', 'ME

As a founding member of the Cloud Security Alliance (CSA), Zscaler, along with alliance partners CrowdStrike and Okta, announced the formation of the Zero Trust Advancement Center to bring together existing research and education projects at CSA that will be disseminated online and through its global network chapters.	NER:[('Cloud Security Alliance', 'MISC'), ('Zscaler', 'ORG'), ('CrowdStrike', 'ORG'), ('Okta', 'ORG'), ('announced', 'CHG'), ('Zero Trust', 'MISC')]###RELATIONS:[]
SC*** Financial Outlook.	NER:[('Outlook', 'FC')]###RELATIONS:[]
NOPAD*** For the fourth quarter of fiscal 2022, we expect:.	NER:[('fourth quarter', 'CALENDAR'), ('2022', 'YEAR'), ('expect', 'FC')]###RELATIONS:[]
PG*** Total revenue of $304 million to $306 million.	NER:[('revenue', 'METRIC'), ('$304 million', 'MONEY'), ('$306 million', 'MONEY')]###RELATIONS:['METRIC:REVENUE,METRICTYPE:REGULAR,MONEY:$304MN,RANGEM1:$306MN,RELATION:KVRANGE']
PG*** Non-GAAP income from operations of $33 million to $34 million.	NER:[

In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

Downloading:   0%|          | 0.00/662 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.92G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

In [3]:
inputs = tokenizer("A step by step recipe to make bolognese pasta:", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Toss the pasta with the sauce, then add the meat and toss again.']


In [4]:
inputs = tokenizer("Get the Named entity from this sentence: Total revenue is expected to be between $115.0 million and $117.0 million, representing a year-over-year increase of 24% to 27%.", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Total revenue']


In [5]:
advanced_ner = """Microsoft Corporation is a company that makes computer software and video games. Bill Gates and Paul Allen founded the company in 1975 
[Company]: Microsoft, [Founded]: 1975, [Founders]: Bill Gates, Paul Allen 
 
Amazon.com, Inc., known as Amazon , is an American online business and cloud computing company. It was founded on July 5, 1994 by Jeff Bezos 
[Company]: Amazon, [Founded]: 1994, [Founders]: Jeff Bezos 
 
Apple Inc. is a multinational company that makes personal computers, mobile devices, and software. Apple was started in 1976 by Steve Jobs and Steve Wozniak."""
inputs = tokenizer(advanced_ner, return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['[Company]: Apple, [Founded]: 1976, [Founders]:']


In [6]:
report_ner = """"Revenue is $503MN up 77% year over year
[Metric]: Revenue, [Money]: $503MN, [PCT]: 77%, [CALENDAR]: year over year

Revenue is $603MN up 87% year over year
[Metric]: Revenue, [Money]: $603MN, [PCT]: 87%, [CALENDAR]: year over year

Revenue is $63MN down 12% year over year.
"""
inputs = tokenizer(report_ner, return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['"Revenue is $503MN up 77% year over year [Metric]:']
